In [1]:
from selenium import webdriver
from time import sleep

In [2]:
web = webdriver.Chrome('./chromedriver.exe')

In [3]:
web.get('https://www.pixiv.net/ranking.php?mode=daily&content=illust&date=20190201')

In [4]:
get_link(web, 9)

NameError: name 'get_link' is not defined

In [5]:
url_list = dict()
def get_link(driver, count):
    try:
        url = driver.find_element_by_xpath("""//*[@id="{}"]/div[2]/a""".format(count))\
            .get_attribute('href')\
            .split('t_id=')[1]\
            .split('&')[0]
    except Exception:
        return ''
    if url in url_list.keys():
        return ''
    url_list[url] = True
    return url

In [6]:
def scrol(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight); return 10")

In [7]:
def one_day(driver, download_image_count):
    #scrol(driver)
    #sleep(5)
    page_ids = []
    for i in range(download_image_count):
        link = get_link(driver, i+1)
        if not link:
            continue
        page_ids.append(link)
    return page_ids

In [8]:
def next_date(now):
    day = get_day(now)
    mon = get_mon(now)
    year = get_year(now)
    if day > 28:
        day = 1
        if mon > 12:
            mon = 1
            year += 1
        else:
            mon += 1
    else:
        day += 1
    return "{year}%02d%02d".format(year=year) % (mon, day)

def get_day(now):
    return int(now[-2:])

def get_mon(now):
    return int(now[-4:6])

def get_year(now):
    return int(now[:4])

In [9]:
def driver_next_day(driver, now):
    try:
        driver.get('https://www.pixiv.net/ranking.php?mode=daily&content=illust&date='+next_date(now))
    except TimeoutException:
        pass
    return next_date(now)

In [10]:
def get_image(driver, illust):
    driver.get('https://www.pixiv.net/member_illust.php?mode=medium&illust_id='+illust)
    

In [11]:
def get_image_url(driver, id):
    id = int(id)
    driver.get('https://www.pixiv.net/member_illust.php?mode=medium&illust_id=%d'%id)
    try:
        pic = driver.find_element_by_xpath('//*[@id="wrapper"]/div/div[2]/div/div/div[1]/div[2]/a/img')
    except Exception:
        try:
            pic = driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/div/main/section/div[1]/figure/div[1]/div[2]/div[2]/a/img')
        except Exception:
            try:
                pic = driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/div/main/section/div[1]/figure/div[1]/div/div/a/img')
            except Exception:
                return None
    return pic.get_attribute('src')

In [15]:
all_count = 0
now = "20181220"
for i in range(20):
    now = driver_next_day(web, now)
    links = one_day(web, 35)
    count = 0
    for link in links:
        url = get_image_url(web, link)
        if not url:
            continue
        web.get(url)
        web.find_element_by_xpath('/html/body/img')
        web.get_screenshot_as_file('./Datas/crol/{now}-{count}.jpg'.format(now=now, count=count))
        count += 1
        all_count += 1
        print(all_count)

C:\Users\haho6\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:1019: UserWarning: name used for saved screenshot does not match file type. It should end with a `.png` extension
  "type. It should end with a `.png` extension", UserWarning)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194


TimeoutException: Message: timeout
  (Session info: chrome=72.0.3626.121)
  (Driver info: chromedriver=2.34.522940 (1a76f96f66e3ca7b8e57d503b4dd3bccfba87af1),platform=Windows NT 10.0.17134 x86_64)


In [77]:
all_count = 0
now = "20181220"
for i in range(20):
    now = driver_next_day(web, now)
    links = one_day(web, 35)
    count = 0
    for link in links:
        url = get_image_url(web, link)
        if not url:
            continue
        print(url)

<selenium.webdriver.remote.webelement.WebElement (session="8c7057bda4b4d45dd41004a4f9707d8b", element="0.6460323824228544-1")>

AttributeError: 'list' object has no attribute 'keys'

{'PHPSESSID': '927558e105417a07d24c8a1bed487439',
 'p_ab_d_id': '1284032605',
 'p_ab_id': '9',
 'p_ab_id_2': '6',
 'first_visit_datetime_pc': '2019-03-05+15%3A12%3A51',
 'yuid_b': 'QFk4U1k'}

<Response [200]>

<Response [403]>